In [6]:
from datadog import initialize, api

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:


# Replace with your keys
API_KEY = os.getenv("DD_API_KEY")
APP_KEY = os.getenv("DD_APP_KEY")

options = {
    "api_key": API_KEY,
    "app_key": APP_KEY
}

# Initialize the client
initialize(api_key=API_KEY, app_key=APP_KEY)

# Fetch all monitors
monitors = api.Monitor.get_all()

print(monitors)
print(f"Total monitors: {len(monitors)}\n")

# for monitor in monitors:
#     print(f"ID: {monitor['id']}")
#     print(f"Name: {monitor['name']}")
#     print(f"Type: {monitor['type']}")
#     print(f"Overall State: {monitor['overall_state']}")
#     print("-" * 50)


{'errors': ['Forbidden']}
Total monitors: 1



In [1]:
import pandas as pd

In [2]:
sample_data = [
    {
        "conversation_id": "CONV-001",
        "message_id": "MSG-001",
        "from": "alice@company.com",
        "to": "bob@company.com",
        "subject": "Project Timeline",
        "body": "Hi Bob, can you share the updated project timeline?",
        "sent_time": "2024-11-01 09:15"
    },
    {
        "conversation_id": "CONV-001",
        "message_id": "MSG-002",
        "from": "bob@company.com",
        "to": "alice@company.com",
        "subject": "RE: Project Timeline",
        "body": "Hi Alice, attaching the updated timeline. Let me know your thoughts.",
        "sent_time": "2024-11-01 10:02"
    },
    {
        "conversation_id": "CONV-001",
        "message_id": "MSG-003",
        "from": "alice@company.com",
        "to": "bob@company.com",
        "subject": "RE: Project Timeline",
        "body": "Thanks Bob. This works for me.",
        "sent_time": "2024-11-01 10:20"
    },
    {
        "conversation_id": "CONV-002",
        "message_id": "MSG-004",
        "from": "manager@company.com",
        "to": "team@company.com",
        "subject": "Weekly Status Update",
        "body": "Please share your weekly status updates by EOD Friday.",
        "sent_time": "2024-11-02 08:30"
    }
]

df = pd.DataFrame(sample_data)

# Save to Excel for testing
df.to_excel("mock_outlook_conversations.xlsx", index=False)

In [3]:
def extract_conversation_from_excel(
    excel_path: str,
    conversation_id: str,
    sheet_name: str = 0
):
    """
    Extract and return an email conversation (mail thread)
    based on a conversation_id from an Excel file.

    Parameters
    ----------
    excel_path : str
        Path to the Excel file
    conversation_id : str
        Unique conversation/thread ID
    sheet_name : str or int, optional
        Sheet name or index (default is first sheet)

    Returns
    -------
    pd.DataFrame
        Emails in the conversation, ordered by sent_time
    """

    # Load Excel
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    # Filter conversation
    convo_df = df[df["conversation_id"] == conversation_id].copy()

    if convo_df.empty:
        raise ValueError(f"No conversation found for ID: {conversation_id}")

    # Ensure proper ordering
    if "sent_time" in convo_df.columns:
        convo_df["sent_time"] = pd.to_datetime(convo_df["sent_time"])
        convo_df = convo_df.sort_values("sent_time")

    return convo_df.reset_index(drop=True)

In [4]:
def print_conversation(convo_df: pd.DataFrame):
    for _, row in convo_df.iterrows():
        print("-" * 60)
        print(f"From   : {row['from']}")
        print(f"To     : {row['to']}")
        print(f"Sent   : {row['sent_time']}")
        print(f"Subject: {row['subject']}\n")
        print(row['body'])
        print()

In [5]:
conversation = extract_conversation_from_excel(
    excel_path="mock_outlook_conversations.xlsx",
    conversation_id="CONV-001"
)

print_conversation(conversation)

------------------------------------------------------------
From   : alice@company.com
To     : bob@company.com
Sent   : 2024-11-01 09:15:00
Subject: Project Timeline

Hi Bob, can you share the updated project timeline?

------------------------------------------------------------
From   : bob@company.com
To     : alice@company.com
Sent   : 2024-11-01 10:02:00
Subject: RE: Project Timeline

Hi Alice, attaching the updated timeline. Let me know your thoughts.

------------------------------------------------------------
From   : alice@company.com
To     : bob@company.com
Sent   : 2024-11-01 10:20:00
Subject: RE: Project Timeline

Thanks Bob. This works for me.

